In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import json
import base64
def decode_jwt_payload(jwt):
    # Tách phần payload của JWT
    payload = jwt.split(".")[1]
    
    # Chuyển Base64-URL thành Base64 chuẩn
    payload += "=" * (-len(payload) % 4)  # Thêm padding nếu cần
    decoded_bytes = base64.urlsafe_b64decode(payload)  # Giải mã Base64
    
    # Chuyển bytes thành chuỗi JSON
    decoded_str = decoded_bytes.decode("utf-8")
    
    # Parse JSON thành dictionary
    return json.loads(decoded_str)

user = input("Enter your username: ")
passs = input("Enter your password: ")
os.system('cls')

login_data = {
    'username': user,
    'password': passs,
    '_eventId': 'submit',
    'submit': 'Login',
}


In [2]:

with requests.Session() as S:
    S.cookies.clear()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36 Edg/126.0.0.0'
    }

    # Login to MyBK
    login_url = 'https://sso.hcmut.edu.vn/cas/login?service=https%3A%2F%2Fmybk.hcmut.edu.vn%2Fapp%2Flogin%2Fcas'
    home_url = 'https://mybk.hcmut.edu.vn/app/'
    r = S.get(login_url, headers=headers)
    # print(r.content)
    soup = BeautifulSoup(r.content, 'html5lib')
    login_data['lt'] = soup.find('input', attrs={'name': 'lt'})['value']
    login_data['execution'] = soup.find(
        'input', attrs={'name': 'execution'})['value']
    r = S.post(login_url, data=login_data)
    if r.url == home_url:
        print("Logged in " + user + " successfully")

        uni_records_url = 'https://mybk.hcmut.edu.vn/app/he-thong-quan-ly/sinh-vien/ket-qua-hoc-tap'
        response_records = S.get(uni_records_url)
        soup1 = BeautifulSoup(response_records.content, 'html5lib')
        token = soup1.find('input', attrs={'id': 'hid_Token'})['value']
        
            #print(response_info.json())
        json_data= decode_jwt_payload(token)

        profiles= json_data['profiles']
        person_ID= json.loads(profiles)['personId']

        url_mark= f"https://mybk.hcmut.edu.vn/api/sinh-vien/xem-ket-qua-hoc-tap/v2?mssv={person_ID}&null"
        url_full_info= f"https://mybk.hcmut.edu.vn/api/v1/student/detail-info-by-code/{person_ID}?null"

        if response_records.status_code == 200:
            print("Accessed records page successfully")

            headers = {
                'Authorization': 'Bearer ' + token,
                'Content-Type': 'application/json'
            }
           # info_url = 'https://mybk.hcmut.edu.vn/api/v1/student/detail-info/203850?null'
            #info_url = 'https://mybk.hcmut.edu.vn/api/sinh-vien/ket-qua-hoc-tap/danh-sach-diem-mon-hoc-xet-tuong-duong/v1?null'
            info_url= 'https://mybk.hcmut.edu.vn/api/sinh-vien/ket-qua-hoc-tap/danh-sach-diem-khac/v1?null'
            
            response_info = S.get(info_url, headers=headers)

            profiles = response_info.json()

            response_info = S.get(url_full_info, headers=headers)
            full_info = response_info.json()

            response_info = S.get(url_mark, headers=headers)
            mark =response_info.json()

            with open(f"{person_ID}_profiles.json", "w", encoding="utf-8") as f:
                json.dump(profiles, f, ensure_ascii=False, indent=4)

            with open(f"{person_ID}_infor.json", "w", encoding="utf-8") as f:
                json.dump(full_info, f, ensure_ascii=False, indent=4)

            with open(f"{person_ID}_mark.json", "w", encoding="utf-8") as f:
                json.dump(mark, f, ensure_ascii=False, indent=4)

            if response_info.status_code == 200:
                print("Accessed data info successfully")
                # data = response_info.json()['data']
                # info = {
                #     'Full Name': data['lastName'] + ' ' + data['firstName'],
                #     'Student ID': data['code'],
                #     'Email': data['orgEmail'],
                #     'Department': data['trainingManagementDep']['nameVi'],
                #     'Major': data['major']['nameVi'],
                #     'Class': data['classCode']}
                # print(info)
            else:
                print("Failed to access data info")
        else:
            print("Failed to access records page")
    else:
        print("Failed to login")


Logged in anh.trantnma2004 successfully
Accessed records page successfully
Accessed data info successfully
